In [1]:
import arcpy
import csv
import json
import os
import geopandas as gpd
import matplotlib.pyplot as plt

#arcpy.env.workspace = r'data'

In [2]:
os.getcwd()

'c:\\Users\\mgavil3\\Box\\CursosLSU\\GISProgramming4057\\Projects\\P1'

In [2]:
with open("data/no_tax.json", "r") as json_data:
    study_area = json.load(json_data)
    # study_area # as you can see, it is not printing the json_data

In [3]:
column_names = [item["name"] for item in study_area["meta"]["view"]["columns"]]
values = study_area["data"]
# gdf = gpd.GeoDataFrame(values,columns=[column_names])

In [4]:
# just to make sure the length is changing
print(len(values[0]))

for row in values:
    arcgis_geom = arcpy.FromWKT(row[8])
    row.append(arcgis_geom)

print(len(values[0]))

14
15


## Create feature class and insert field names

In [74]:
workspace = r'C:\Users\mgavil3\Box\CursosLSU\GISProgramming4057\Projects\P1'
fc_name = 'no_tax_fc.shp'
output_folder = os.path.join(workspace, 'output')
fc_full_name = os.path.join(output_folder, fc_name)
if arcpy.Exists(fc_full_name):
    arcpy.management.Delete(fc_full_name)
arcpy.management.CreateFeatureclass(
                                    out_name = fc_name,
                                    out_path = output_folder,
                                    geometry_type = 'POLYGON',
                                    spatial_reference = 4326
                                    )
desc = arcpy.da.Describe(fc_full_name)
for field in desc['fields']:
    print(field.name)

FID
Shape
Id


In [75]:
fields = study_area["meta"]["view"]["columns"]
fields = [field['name'] for field in fields if field['name'] != 'the_geom']
field_type = ['TEXT','TEXT','LONG','LONG','TEXT','LONG','TEXT','TEXT','TEXT','TEXT','TEXT','TEXT','TEXT']# not including the geom nor the ARGIS Polygon]
fields[1] = 'uid'
fields[9] = 'uid2'
fields = [field.replace(' ','_') for field in fields]
fields = [field.replace('.','_') for field in fields]
fields = [field[:10] if len(field)>10 else field for field in fields]
fields

['sid',
 'uid',
 'position',
 'created_at',
 'created_me',
 'updated_at',
 'updated_me',
 'meta',
 'OBJECTID',
 'uid2',
 'Cluster_Le',
 'Shape_STAr',
 'Shape_STLe']

In [76]:
for idx, field in enumerate( fields):
    # print(field_name)
    arcpy.management.AddField(in_table = fc_full_name, field_name = field, field_type = field_type[idx])

## Adding data to feature class

In [77]:
fields.append('SHAPE@')
len(fields)

14

In [122]:
with arcpy.da.InsertCursor(fc_full_name,field_names=fields) as cursor:
    for row in study_area['data']:
        inserting_row = []
        for idx2, value in enumerate(row):
            if idx2 == 8:
                continue
            if (value == None):
                value = ''
            inserting_row.append(value)
        cursor.insertRow(inserting_row)